In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import keras as kr

In [3]:
# DF de entrenamiento, con todas las variables y la columna tier completa
df_train: pd.DataFrame = pd.DataFrame(
    columns=("value", "typeA", "typeB", "classA", "classB", "tier")
)
df_train.loc[0] = [50000, 1, 0, 0, 1, 1]
df_train.loc[1] = [49000, 1, 0, 0, 1, 2]
df_train.loc[2] = [40000, 1, 0, 1, 0, 2]
df_train.loc[3] = [39000, 0, 1, 0, 1, 3]
df_train.loc[4] = [30000, 1, 0, 1, 0, 3]
df_train.loc[5] = [21000, 1, 0, 0, 1, 3]
df_train.loc[6] = [20000, 0, 1, 1, 0, 4]
df_train.loc[7] = [11000, 0, 1, 0, 1, 4]
df_train.loc[8] = [10000, 1, 0, 1, 0, 5]
df_train.loc[9] = [1000, 1, 0, 1, 0, 5]


# DF de test, con todas las variables y la columna tier completa
df_test: pd.DataFrame = pd.DataFrame(
    columns=("value", "typeA", "typeB", "classA", "classB", "tier")
)
df_test.loc[0] = [80000, 1, 0, 0, 1, 1]
df_test.loc[1] = [37000, 1, 0, 0, 1, 3]
df_test.loc[2] = [32000, 1, 0, 0, 1, 3]
df_test.loc[3] = [18000, 1, 0, 0, 1, 4]
df_test.loc[4] = [1000, 1, 0, 0, 1, 5]
df_test.loc[5] = [200, 1, 0, 0, 1, 5]
df_test.loc[6] = [45000, 1, 0, 0, 1, 2]

## Preparación de datos

In [4]:
# Separar características y etiquetas

X_train: pd.DataFrame = df_train.drop('tier', axis=1)
y_train: pd.Series = df_train['tier']

X_test: pd.DataFrame = df_test.drop('tier', axis=1)
y_test: pd.Series = df_test['tier']

# Normalizar los datos
scaler: StandardScaler = StandardScaler()
X_train: pd.DataFrame = scaler.fit_transform(X_train)
X_test: pd.DataFrame = scaler.transform(X_test)

# Convertir etiquetas a categorías
y_train = kr.utils.to_categorical(y_train - 1, num_classes=5)
y_test = kr.utils.to_categorical(y_test - 1, num_classes=5)

## Definición del modelo

In [5]:
# Definir el modelo
model: kr.models.Sequential = kr.models.Sequential([
    kr.layers.Input(shape=(X_train.shape[1],)),
    kr.layers.Dense(128, activation='relu'),
    kr.layers.Dense(64, activation='relu'),
    kr.layers.Dense(32, activation='relu'),
    kr.layers.Dense(5, activation='softmax')
])

# Compilar el modelo
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

## Entrenamiento y evaluación del modelo

In [6]:
# Aquí te he hecho un entrenamiento con unos parámetros a ojo, pero es solo 
# una prueba.

# Lo suyo sería que entrenaras con distintos epochs, validation_splits y 
# batch_size para encontrar el "punto dulce" de precisión del modelo.

# Entrenar el modelo
history = model.fit(
    x=X_train, y=y_train,
    epochs=1000, validation_split=0.2, batch_size=1,
    verbose=0
)

# Evaluar el modelo
loss, accuracy = model.evaluate(x=X_test, y=y_test, verbose=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.5714 - loss: 48.6358


## Predicción de tiers

In [7]:
# DF con datos pero sin valor de tiers
df_score: pd.DataFrame = pd.DataFrame(
    columns=("value", "typeA", "typeB", "classA", "classB",)
)

df_score.loc[0] = [800000, 1, 0, 0, 1]
df_score.loc[1] = [45500, 1, 0, 1, 0]
df_score.loc[2] = [27900, 0, 1, 0, 1]
df_score.loc[3] = [12000, 0, 1, 1, 0]
df_score.loc[4] = [440, 1, 0, 0, 1]

In [ ]:
# Normalizar los datos de df_score
X_score = df_score.values
X_score = scaler.transform(X=X_score)

# Predecir los tiers
predictions = model.predict(x=X_score)
predicted_tiers = predictions.argmax(axis=1) + 1

# Añadir las predicciones a df_score
df_score['predicted_tier'] = predicted_tiers


In [9]:
df_score

,value,typeA,typeB,classA,classB,predicted_tier
0,800000,1,0,0,1,1
1,45500,1,0,1,0,2
2,27900,0,1,0,1,3
3,12000,0,1,1,0,4
4,440,1,0,0,1,3
